In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from pathlib import Path
from urllib.parse import urlparse, unquote
import requests
from tqdm import tqdm
import fitz  
import zipfile

In [19]:
def safe_filename(name: str) -> str:
    """Limpia nombres raros para guardar en disco."""
    name = unquote(name)
    name = re.sub(r"[^\w\-.() ]+", "_", name, flags=re.UNICODE)
    name = re.sub(r"\s+", " ", name).strip()
    return name


def filename_from_url(url: str) -> str:
    p = urlparse(url)
    base = os.path.basename(p.path) or "archivo"
    base = safe_filename(base)
    return base


def download_file(url: str, out_path: Path, timeout=60):
    """Descarga con streaming y reintentos simples."""
    out_path.parent.mkdir(parents=True, exist_ok=True)

    # Si ya existe y pesa algo, no lo bajes de nuevo
    if out_path.exists() and out_path.stat().st_size > 0:
        return "skip"

    headers = {"User-Agent": "Mozilla/5.0 (compatible; bulk-downloader/1.0)"}

    with requests.get(url, stream=True, timeout=timeout, headers=headers) as r:
        r.raise_for_status()
        total = int(r.headers.get("Content-Length", 0))
        tmp = out_path.with_suffix(out_path.suffix + ".part")

        with open(tmp, "wb") as f, tqdm(
            total=total if total > 0 else None,
            unit="B",
            unit_scale=True,
            desc=out_path.name,
            leave=False,
        ) as pbar:
            for chunk in r.iter_content(chunk_size=1024 * 256):
                if chunk:
                    f.write(chunk)
                    if total > 0:
                        pbar.update(len(chunk))

        tmp.replace(out_path)

    return "ok"


def extract_links_from_pdf(pdf_path: Path):
    """Extrae URIs de enlaces embebidos en el PDF."""
    doc = fitz.open(pdf_path)
    links = []
    for page in doc:
        for link in page.get_links():
            uri = link.get("uri")
            if uri:
                links.append(uri)
    doc.close()
    # quitar duplicados preservando orden
    seen = set()
    uniq = []
    for u in links:
        if u not in seen:
            seen.add(u)
            uniq.append(u)
    return uniq


def main(index_pdf_url: str, out_dir="sarampion_2025"):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    index_pdf_path = out_dir / "indice.pdf"
    print("Descargando índice PDF…")
    download_file(index_pdf_url, index_pdf_path)

    print("Extrayendo links del PDF…")
    links = extract_links_from_pdf(index_pdf_path)
    print(f"Links encontrados: {len(links)}")

    wanted_ext = (".zip", ".csv", ".xlsx", ".xls", ".txt", ".pdf")
    links = [u for u in links if urlparse(u).path.lower().endswith(wanted_ext)]
    print(f"Links tras filtro por extensión: {len(links)}")

    ok = skip = fail = 0
    print("Descargando archivos…")
    for url in tqdm(links, desc="Total", unit="archivo"):
        fname = filename_from_url(url)
        out_path = out_dir / fname
        try:
            status = download_file(url, out_path)
            if status == "ok":
                ok += 1
            elif status == "skip":
                skip += 1
        except Exception as e:
            fail += 1
            print(f"\n[ERROR] {url}\n  -> {e}\n")

    print("\nResumen:")
    print("  descargados:", ok)
    print("  ya existían:", skip)
    print("  fallidos:", fail)
    print("Carpeta:", out_dir.resolve())


if __name__ == "__main__":

    INDEX_PDF_URL = "https://www.gob.mx/cms/uploads/attachment/file/1041077/datos_abiertos_historicos_efe_2025.pdf"
    main(INDEX_PDF_URL, out_dir="datos_semanales_2025")


Descargando índice PDF…
Extrayendo links del PDF…
Links encontrados: 43
Links tras filtro por extensión: 43
Descargando archivos…


Total: 100%|██████████████████████████████| 43/43 [00:00<00:00, 466.55archivo/s]


[ERROR] https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2025/datos_abiertos_efe_0050225.zip
  -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2025/datos_abiertos_efe_0050225.zip


Resumen:
  descargados: 0
  ya existían: 42
  fallidos: 1
Carpeta: /Users/jamc/Desktop/Datos_Sarampión/datos_semanales_2025


Semanas faltantes:

2023  -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2023/datos_abiertos_efe_150523.zip

2024  -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2024/datos_abiertos_efe_280624.zip

2024  -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2024/datos_abiertos_efe_040124.zip

2025 -> 404 Client Error: Not Found for url: https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/efe/historicos/2025/datos_abiertos_efe_0050225.zip


In [14]:
ruta_base = "." 
todos_los_dfs = []  

for carpeta in sorted(os.listdir(ruta_base)):

    if carpeta.startswith("datos_semanales") and os.path.isdir(carpeta):
        ruta_carpeta = os.path.join(ruta_base, carpeta)
        print(f"--> Procesando carpeta: {carpeta}")
        
        for archivo in os.listdir(ruta_carpeta):
            if archivo.endswith(".zip"):
                ruta_zip = os.path.join(ruta_carpeta, archivo)
                
                try:

                    with zipfile.ZipFile(ruta_zip, "r") as z:
 
                        for nombre_archivo_interno in z.namelist():
                            if nombre_archivo_interno.endswith(".csv"):
                                with z.open(nombre_archivo_interno) as f:
                        
                                    df_temporal = pd.read_csv(f, encoding='utf-8') 
                                    todos_los_dfs.append(df_temporal)
                                break 
                except Exception as e:
                    print(f"Error leyendo {archivo}: {e}")


if todos_los_dfs:
    df_final = pd.concat(todos_los_dfs, ignore_index=True)
    

#    df_final.to_csv("datos_concatenados.csv", index=False)
    print(f"Se han unido {len(todos_los_dfs)} archivos.")
    print(f"Dimensiones finales: {df_final.shape}")
else:
    print("No se encontraron archivos CSV para unir.")

--> Procesando carpeta: datos_semanales_2020
--> Procesando carpeta: datos_semanales_2021
--> Procesando carpeta: datos_semanales_2022
--> Procesando carpeta: datos_semanales_2023
--> Procesando carpeta: datos_semanales_2024
--> Procesando carpeta: datos_semanales_2025
--> Procesando carpeta: datos_semanales_2026
Se han unido 227 archivos.
Dimensiones finales: (462289, 22)


In [15]:
df_final

,FECHA_ACTUALIZACION,ID_REGISTRO,EDAD_ANOS,EDAD_MESES,EDAD_DIAS,SEXO,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INSTITUCION_NOTIF,VACUNACION,EXANTEMA,FIEBRE,COMPLICACIONES,DEFUNCION,DIAGNOSTICO,CRITERIO_DIAGNOSTICO,FECHA_DIAGNOSTICO,ORIGEN_CASO
0,2021-05-03,20729,9,2,21,2,2,2,14,101,...,4,1,1,1.0,2,2,0,0,9999-99-99,5.0
1,2021-05-03,20730,12,7,30,2,2,2,14,81,...,12,1,1,1.0,2,2,3,0,2021-01-09,5.0
2,2021-05-03,20731,7,6,11,1,2,2,14,81,...,12,1,1,1.0,2,2,3,0,2021-01-09,5.0
3,2021-05-03,20732,12,7,30,2,2,2,14,81,...,12,1,1,1.0,2,2,3,0,2021-01-09,5.0
4,2021-05-03,20733,1,5,8,2,2,2,30,21,...,12,2,1,1.0,2,2,3,0,2021-01-17,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462284,05/01/2026,37187,30,7,20,1,2,2,8,50,...,4,1,1,1.0,2,2,1,2,24/06/2025,2.0
462285,05/01/2026,37363,29,7,20,1,2,2,8,50,...,4,2,1,1.0,2,2,1,1,30/06/2025,2.0
462286,05/01/2026,37364,28,8,3,1,2,2,8,50,...,4,2,1,1.0,2,2,1,2,30/06/2025,2.0
462287,05/01/2026,37570,1,9,11,2,2,2,8,50,...,4,1,1,1.0,2,2,3,2,30/06/2025,5.0


In [16]:
df_final = df_final[df_final['DIAGNOSTICO']==1] # sarampión = 1
df_final = df_final.drop_duplicates(subset=['ID_REGISTRO']) # eliminamos ID's duplicados

In [17]:
df_final

,FECHA_ACTUALIZACION,ID_REGISTRO,EDAD_ANOS,EDAD_MESES,EDAD_DIAS,SEXO,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INSTITUCION_NOTIF,VACUNACION,EXANTEMA,FIEBRE,COMPLICACIONES,DEFUNCION,DIAGNOSTICO,CRITERIO_DIAGNOSTICO,FECHA_DIAGNOSTICO,ORIGEN_CASO
2662,2020-12-23,18397,8,5,2,1,2,2,9,14,...,4,1,1,1.0,2,2,1,2,2020-03-03,4.0
2691,2020-12-23,18427,39,4,3,2,2,2,9,5,...,12,2,1,1.0,2,2,1,2,2020-03-05,4.0
2692,2020-12-23,18428,37,0,28,2,2,2,9,5,...,12,2,1,1.0,2,2,1,2,2020-03-05,4.0
2703,2020-12-23,18439,10,10,25,2,2,2,9,5,...,12,2,1,1.0,2,2,1,2,2020-03-05,4.0
2717,2020-12-23,18453,46,5,2,2,2,2,9,5,...,12,2,1,1.0,2,2,1,2,2020-03-05,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460075,05/01/2026,45001,1,6,6,2,1,1,8,36,...,12,2,1,1.0,1,2,1,2,08/12/2025,4.0
460076,05/01/2026,45003,24,1,24,1,1,1,8,36,...,12,2,1,1.0,1,2,1,2,08/12/2025,4.0
460078,05/01/2026,45245,53,2,3,2,2,2,8,36,...,12,2,1,1.0,2,2,1,2,15/12/2025,4.0
461132,05/01/2026,44540,30,6,29,1,2,2,8,17,...,12,2,1,1.0,2,2,1,2,26/11/2025,4.0


In [18]:
def normaliza_fecha_col(df, col, slash_format="%d/%m/%Y"):
    s = df[col].astype(str).str.strip()

    m_iso = s.str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)
    m_slash = s.str.contains(r"/", na=False)

    dt = pd.Series(pd.NaT, index=df.index, dtype="datetime64[ns]")
    dt[m_iso]   = pd.to_datetime(s[m_iso],   format="%Y-%m-%d", errors="coerce")
    dt[m_slash] = pd.to_datetime(s[m_slash], format=slash_format, errors="coerce")

    rest = dt.isna() & s.ne("nan")
    dt[rest] = pd.to_datetime(s[rest], errors="coerce", dayfirst=True)

    df[col] = dt.dt.strftime("%Y-%m-%d")
    return df

In [19]:
df_final = normaliza_fecha_col(df_final, "FECHA_ACTUALIZACION", slash_format="%d/%m/%Y")
df_final = normaliza_fecha_col(df_final, "FECHA_DIAGNOSTICO",    slash_format="%d/%m/%Y")

print("nulos en FECHA_ACTUALIZACION:", df_final["FECHA_ACTUALIZACION"].isna().sum())
print("nulos en FECHA_DIAGNOSTICO:", df_final["FECHA_DIAGNOSTICO"].isna().sum())


nulos en FECHA_ACTUALIZACION: 0
nulos en FECHA_DIAGNOSTICO: 0


In [20]:
df_final = df_final[['FECHA_DIAGNOSTICO', 'ID_REGISTRO', 'EDAD_ANOS', 'EDAD_MESES',
       'EDAD_DIAS', 'SEXO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'ENTIDAD_UM_NOTIF', 'MUNICIPIO_UM_NOTIF', 'ENTIDAD_RES',
       'MUNICIPIO_RES', 'INSTITUCION_NOTIF', 'VACUNACION', 'EXANTEMA',
       'FIEBRE', 'COMPLICACIONES', 'DEFUNCION', 'DIAGNOSTICO',
       'CRITERIO_DIAGNOSTICO','FECHA_ACTUALIZACION',  'ORIGEN_CASO']].sort_values('FECHA_DIAGNOSTICO')

In [21]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7107 entries, 2662 to 446197
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   FECHA_DIAGNOSTICO     7107 non-null   object 
 1   ID_REGISTRO           7107 non-null   int64  
 2   EDAD_ANOS             7107 non-null   int64  
 3   EDAD_MESES            7107 non-null   int64  
 4   EDAD_DIAS             7107 non-null   int64  
 5   SEXO                  7107 non-null   int64  
 6   HABLA_LENGUA_INDIG    7107 non-null   int64  
 7   INDIGENA              7107 non-null   int64  
 8   ENTIDAD_UM_NOTIF      7107 non-null   int64  
 9   MUNICIPIO_UM_NOTIF    7107 non-null   int64  
 10  ENTIDAD_RES           7107 non-null   int64  
 11  MUNICIPIO_RES         7107 non-null   int64  
 12  INSTITUCION_NOTIF     7107 non-null   int64  
 13  VACUNACION            7107 non-null   int64  
 14  EXANTEMA              7107 non-null   int64  
 15  FIEBRE               

In [22]:
df_final['FECHA_DIAGNOSTICO'] = pd.to_datetime(df_final['FECHA_DIAGNOSTICO'])

In [23]:
df_final

,FECHA_DIAGNOSTICO,ID_REGISTRO,EDAD_ANOS,EDAD_MESES,EDAD_DIAS,SEXO,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INSTITUCION_NOTIF,VACUNACION,EXANTEMA,FIEBRE,COMPLICACIONES,DEFUNCION,DIAGNOSTICO,CRITERIO_DIAGNOSTICO,FECHA_ACTUALIZACION,ORIGEN_CASO
2662,2020-03-03,18397,8,5,2,1,2,2,9,14,...,4,1,1,1.0,2,2,1,2,2020-12-23,4.0
2691,2020-03-05,18427,39,4,3,2,2,2,9,5,...,12,2,1,1.0,2,2,1,2,2020-12-23,4.0
2692,2020-03-05,18428,37,0,28,2,2,2,9,5,...,12,2,1,1.0,2,2,1,2,2020-12-23,4.0
2703,2020-03-05,18439,10,10,25,2,2,2,9,5,...,12,2,1,1.0,2,2,1,2,2020-12-23,4.0
2717,2020-03-05,18453,46,5,2,2,2,2,9,5,...,12,2,1,1.0,2,2,1,2,2020-12-23,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446349,2026-01-20,47740,14,5,13,2,2,2,25,9,...,12,2,1,1.0,2,2,1,2,2026-01-20,4.0
446332,2026-01-20,47723,2,2,5,1,2,2,25,9,...,12,2,1,1.0,2,2,1,2,2026-01-20,4.0
446329,2026-01-20,47720,29,6,10,1,2,2,25,9,...,12,2,1,1.0,2,2,1,2,2026-01-20,4.0
446241,2026-01-20,47631,5,11,13,1,2,2,25,9,...,12,2,1,1.0,2,2,1,2,2026-01-20,4.0


In [24]:
df_final.to_csv("base_sarampion_confirmados.csv", index=False, encoding='utf-8')